In [1]:
import pandas as pd
import unicodedata
import re
import nltk

In [2]:
from google.colab import drive
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import nltk

# Loading data via google drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_df = pd.read_csv(r"/content/drive/My Drive/data/dataset-metadata-with-lyrics.csv")
data_df = pd.DataFrame(data_df)
data_df

,song_id,song_name,Artist,Lyrics,Type
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේ\r\nඔබ දුරයි නෙත...,Calm
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේ\r\n\r\nබඹරු නටන හැන්දෑවේ…...,Calm
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,\r\nසමනළ මුදුනේ සිරිපද සිඹ සිඹ\r\nඋදා ඉරක් පාය...,Calm
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැව\r\nපමණි මට අද ශේ...,Calm
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා...\r\nඋදා හිරු එළියේ......,Calm
...,...,...,...,...,...
157,S 48,Dangakaaree Man,Indrani Wijebandara,දඟකාරී මං මද්දුම දෝණි\nඔබට අකීකරු නෑ අම්මේ\nදඟ...,Sad
158,S 49,Edaa Wage Adath,Somathilaka Jayamaha,එදා වාගේ අදත් පොද වැස්ස වැටෙනවා\r\nමල් පිපුණු ...,Sad
159,S 50,Paluwa Dewanath Karala,Gunadasa Kapuge,පාළුව දෙවනත් කරලා\r\nනාඩන් උලලේනෝ\r\nකාටත් නපු...,Sad
160,S 51,Pawena Nil Walawe,W. D. Amaradeva,පාවෙනා නිල් වලාවේ\r\nනදී තලාවේ\r\nරිදී ජලාසේ\r...,Sad


In [4]:
def lineBreakReplace(text):
    return text.replace("\r\n", " ")

In [5]:
data_df['Lyrics'] =data_df.apply(lambda x: lineBreakReplace(x.Lyrics), axis=1)

In [6]:
def removeEscapSequences(text):
    return "".join(ch for ch in text if unicodedata.category(ch)[0]!="C")

In [7]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [8]:
def remove_digits(text):
    return ''.join([i for i in text if not i.isdigit()])

In [9]:
data_df['Lyrics'] =data_df.apply(lambda x: removeEscapSequences(x.Lyrics), axis=1)

In [10]:
def removeHtmlTags(text):
    p = re.compile(r'<.*?>')
    return p.sub('', text)

In [11]:
data_df['Lyrics'] = data_df.apply(lambda x: removeHtmlTags(x.Lyrics), axis=1)

In [12]:
data_df.head()

,song_id,song_name,Artist,Lyrics,Type
0,C 01,Kaulu Piyan Path Wahanna,Kasun Kalhara,කවුළු පියන්පත් වහන්න සඳළුතලාවේ ඔබ දුරයි නෙතට ම...,Calm
1,C 02,Ruk Aththana Mala Mudune\n,Nanda Malini,රුක් අත්තන මල මුදුනේ බඹරු නටන හැන්දෑවේ… සැදෑ ...,Calm
2,C 03,Samanala Mudune,H R Jothipala and Latha Walpola,සමනළ මුදුනේ සිරිපද සිඹ සිඹ උදා ඉරක් පායයි සැන...,Calm
3,C 04,Niwan Dutu Himi,Victor Rathnayaka,නිවන් දුටු හිමි රැවන් පිළිරැව පමණි මට අද ශේෂ ව...,Calm
4,C 05,Suwanda Danee Danee Danenawa,Rookantha Gunathilake,සුවඳ දැනී දැනී දැනෙනවා... උදා හිරු එළියේ... අළ...,Calm


In [13]:
def removeSpecialCharacters(sentence,keep_apostrophes=True):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]'
        filtered_sentence = re.sub(PATTERN, r'', sentence)
        return filtered_sentence
    return sentence

In [14]:
def replace_full_stops(sentence):
    return sentence.replace("."," ")

In [15]:
data_df['Lyrics'] = data_df.apply(lambda x: removeSpecialCharacters(x.Lyrics), axis=1)
data_df['Lyrics'] = data_df.apply(lambda x: replace_full_stops(x.Lyrics), axis=1)

In [16]:
def remove_english_letters(sentence):
    non_english_sentence = re.sub(r'[a-zA-Z]', '', sentence)
    return non_english_sentence

In [17]:
data_df['Lyrics']  = data_df['Lyrics'] .apply(lambda x: remove_english_letters(x))

In [18]:
lines = list(open('/content/drive/My Drive/data/stop-words-sinhala.txt',encoding="utf8"))
sinhala_stop_words = [re.sub("\n","",x) for x in lines]

In [19]:
def tokenize_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    return tokens

In [20]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
data_df['token_list']  = data_df['Lyrics'] .apply(lambda x: tokenize_text(x))

In [22]:
def remove_sinhala_stop_words(tokens):
    filtered_tokens = [token for token in tokens if token not in sinhala_stop_words]
    return filtered_tokens

In [23]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_sinhala_stop_words(x.token_list), axis=1)

In [24]:
def remove_two_letter_words(tokenized_words):
    # Use list comprehension to filter out words with length 2
    filtered_words = [word for word in tokenized_words if len(word) > 3]
    return filtered_words

In [25]:
data_df['token_list_after_sw'] = data_df.apply(lambda x: remove_two_letter_words(x.token_list_after_sw), axis=1)

In [26]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_sw), axis=1)

In [27]:
list_of_words = ' '.join(data_df['pre_processed_text']).split()

In [28]:
from collections import Counter
import numpy as np
#most_common_words = [word for word, count in Counter(list_of_words).items() if count in np.arange(50,100)]
unique_words = [word for word, count in Counter(list_of_words).items() if count in [1,2,3]]
all_words_to_remove = unique_words;
def remove_most_common_unique_words(tokens):
    filtered_tokens = [token for token in tokens if token not in all_words_to_remove]
    return filtered_tokens

In [29]:
data_df['token_list_after_common_words'] = data_df.apply(lambda x: remove_most_common_unique_words(x.token_list_after_sw), axis=1)

In [30]:
data_df['pre_processed_text'] = data_df.apply(lambda x: TreebankWordDetokenizer().detokenize(x.token_list_after_common_words), axis=1)

In [31]:
# list_of_words = ' '.join(data_df['pre_processed_text']).split()
# Counter(list_of_words)

In [32]:
data_df['word_count'] = data_df['pre_processed_text'].str.split().str.len()
#data_df.head(40)

In [33]:
def total_unique_words(words):
    return nltk.FreqDist(words)

print("Total number of unique words",total_unique_words(list_of_words))

Total number of unique words <FreqDist with 4290 samples and 12566 outcomes>


In [34]:
def total_words(words):
    return(len(words))
print("Total number of words",total_words(list_of_words))

Total number of words 12566


In [35]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [36]:
#X= data_df['pre_processed_text']
data_df["final_data"] = "__label__"+data_df['Type'] + " " + data_df["pre_processed_text"]
#data_df
#y=label_encoder.fit_transform(data_df['Type'])

In [49]:
train, test = train_test_split(data_df, test_size=0.3)

In [50]:
train.to_csv(r"/content/drive/My Drive/data/train.csv",columns=['final_data'],index=False,header=False)


In [51]:
test.to_csv(r"/content/drive/My Drive/data/test.csv",columns=['final_data'],index=False,header=False)

In [52]:
!pip install fasttext

In [53]:
import fasttext as ft

In [54]:
classifier = ft.train_supervised(input="/content/drive/My Drive/data/train.csv",lr=1.0, epoch=25, wordNgrams=2)

In [55]:
classifier

In [56]:
out= classifier.test(r"/content/drive/My Drive/data/test.csv",k=10)

In [57]:
#numberofsamples  #precision # recall
out

(49, 0.3333333333333333, 1.0)

In [58]:
accuracy= 2 * ((out[1] *out[2]) / (out[1] + out[2]))

In [59]:
accuracy*100

50.0

In [60]:
classifier.predict("දුක දන්නේ වෙන කවුද ඇස් දෙක තරම්ම මම",k=3)

(('__label__Sad', '__label__Calm', '__label__Happy'),
 array([0.84007603, 0.0970296 , 0.06292434]))